In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
archivo = "dataset/Estadisticas_historicas_comercializacion.xlsx"

df_importacion = pd.read_excel(archivo, sheet_name="IMPORTACION", skiprows=6, nrows=289 )
df_consumo = pd.read_excel(archivo, sheet_name="CONSUMO", skiprows=6, nrows=301 ) 
#Aca arriba lo que hicimos fue leer los archivos de excel y saltar las primeras 6 filas que son encabezados y no nos interesan.
#Tambien especificamos el numero de filas que queremos leer, en este caso 289 para importacion y 301 para consumo.

# Renombrar columnas para facilitar
cols_import = df_importacion.columns.tolist()
cols_consumo = df_consumo.columns.tolist()

print("Columnas Importación:")
print(cols_import)

print("\nColumnas Consumo:")
print(cols_consumo)

Columnas Importación:
['Fecha', 'Aceites lubricantes', 'Asfalto ', 'Bunker', 'Ceras', 'Combustible turbo jet', 'Diesel bajo azufre', 'Diesel ultra bajo azufre', 'Gas licuado de petróleo', 'Gasolina de aviación', 'Gasolina regular', 'Gasolina superior', 'Grasas lubricantes', 'Kerosina', 'Mezclas oleosas', 'Naftas', 'Petcoke', 'Petróleo crudo', 'Solventes', 'Butano', 'Diesel alto azufre', 'MTBE', 'Orimulsión', 'Petróleo reconstituido', 'Total importación']

Columnas Consumo:
['Fecha', 'Aceites lubricantes', 'Asfalto', 'Bunker', 'Ceras', 'Combustible turbo jet', 'Diesel bajo azufre', 'Diesel ultra bajo azufre', 'Gas licuado de petróleo', 'Gas natural', 'Gasolina de aviación', 'Gasolina regular', 'Gasolina superior', 'Grasas lubricantes', 'Kerosina', 'Mezclas\noleosas', 'Naftas', 'PetCoke', 'Petróleo crudo', 'Solventes', 'Butano', 'Diesel alto azufre', 'Orimulsión', 'Total']


Bien logramos identificar las columnas importantes ahora vamos a solo a agarrar las que nos interesan. 

In [46]:
df_importacion = df_importacion[['Fecha', 'Gasolina regular', 'Gasolina superior', 'Diesel alto azufre']]
df_consumo = df_consumo[['Fecha', 'Gasolina regular', 'Gasolina superior', 'Diesel alto azufre']]


Vamos a analizar como viene formateada la fecha.. 

In [47]:
print(df_importacion['Fecha'].head())

0   2001-01-01
1   2001-02-01
2   2001-03-01
3   2001-04-01
4   2001-05-01
Name: Fecha, dtype: datetime64[ns]


MMm es tipo object asi que la vamos a transformar a datetime

In [48]:
df_importacion['Fecha'] = pd.to_datetime(df_importacion['Fecha'])
df_consumo['Fecha'] = pd.to_datetime(df_consumo['Fecha'])

In [49]:
print(df_importacion['Fecha'].dtype)


datetime64[ns]


Ya que trabajaremos series de tiempo, conviene que Fecha sea el índice del dataframe:

In [50]:
df_importacion.set_index('Fecha', inplace=True)
df_consumo.set_index('Fecha', inplace=True)
